From the TCR3d Database the TCR Alpha/Beta complexes are downloaded. These are TCR alpha and beta chains renumbered by AHO scheme. 
The naming shows the chain names (e.g. 3sxa_NO_aho.pdb)
Fist all chains are renamed as a and b.

In [ ]:
from Bio.PDB import PDBList, PDBIO, PDBParser
import os

pdbl = PDBList()

io = PDBIO()
parser = PDBParser()
directory="/home/lilian/TCR_Graphs/TCR3d_dataset_061023/tcr_structures/ab_chain"

for file in os.listdir(directory):
    filename = os.path.join(directory, file)
    if filename.endswith(".pdb"): 
        structure = parser.get_structure('file', filename)
        for model in structure:
            counter=0
            for chain in model:
                counter+=1
                old_name = chain.get_id()
                if counter == 1:
                    new_name = "A"
                elif counter == 2:
                    new_name = "B"
                #print(f"renaming chain {old_name} to {new_name}")
                chain.id = new_name
                
        io.set_structure(structure)
        io.save(filename)